### This notebook provide the parameter and FLOPs statics of SNN/LIAF-SNN/3D-CNN/2D-CNN

In [1]:
from __future__ import print_function
import sys
sys.path.append("..")
import torch.distributed as dist 
import torch.nn as nn
import LIAF,LIAFnet
import argparse, pickle, torch, time, os,sys
from importlib import import_module
from LIAFnet.LIAFResNet import *


modules = import_module('LIAFnet.LIAFResNet_34')
config  = modules.Config()
snn34 = LIAFResNet(config)

modules = import_module('LIAFnet.LIAFResNet_18')
config  = modules.Config()
snn18 = LIAFResNet(config)

from LIAFnet.ResNet2D import *
cnn2d18 = resnet18()
cnn2d34 = resnet34()

from LIAFnet.ResNet3D import *

cnn3d18 = resnet18()
cnn3d34 = resnet34()

In [2]:
from LIAF import *
from LIAFnet.LIAFResNet import *
from torch.nn import BatchNorm3d,Conv2d,Linear,Conv3d


time_windows = 8
feature_size = {2:224,64:55,128:28,256:14,512:7,1:224}#chennel:size
feature_size_2 = {64:4,128:2,256:1,512:1}

def cal_LIAFConvCell(m,isliaf=True):
        print(type(m))
        kernel_size = m.kernel.kernel_size[0]
        inchannel =  m.kernel.in_channels
        outchennel = m.kernel.out_channels
        stride = m.kernel.stride[0]
        padding = m.kernel.padding[0]
        n = feature_size[inchannel]
        fp32mut_this = 0
        fp32plus_this = 0
        #Conv
        outsize = math.floor((n + 2 * padding - kernel_size) / stride + 1)
        if isliaf:
            fp32mut_this += inchannel * outchennel * outsize**2*kernel_size**2 
        fp32plus_this += inchannel * outchennel * outsize**2*(kernel_size**2-1)
        #U_update 
        fp32plus_this += outchennel * outsize**2
        #U_spike
        fp32plus_this += outchennel * outsize**2
        fp32mut_this += 2*outchennel * outsize**2
        print('intput:',n,'*',n, 'output:',outsize,'*',outsize)
        return fp32plus_this*time_windows ,fp32mut_this*time_windows

def cal_Temporal_Conv2d(m,isliaf=True):
        print(type(m))
        kernel_size = m.kernel_size[0]
        inchannel =  m.in_channels
        outchennel = m.out_channels
        stride = m.stride[0]
        padding = m.padding[0]
        n = feature_size[inchannel]
        fp32mut_this = 0
        fp32plus_this = 0
        #Conv
        outsize = math.floor((n + 2 * padding - kernel_size) / stride + 1)
        if isliaf:
            fp32mut_this += inchannel * outchennel * outsize**2*kernel_size**2 
        fp32plus_this += inchannel * outchennel * outsize**2*(kernel_size**2-1)
        #U_update 
        fp32plus_this += outchennel * outsize**2
        #U_spike
        fp32plus_this += outchennel * outsize**2
        fp32mut_this += 2*outchennel * outsize**2
        print('intput:',n,'*',n, 'output:',outsize,'*',outsize)
        return fp32plus_this*time_windows, fp32mut_this*time_windows
    
def cal_Conv2d(m):
        print(type(m))
        kernel_size = m.kernel_size[0]
        inchannel =  m.in_channels
        outchennel = m.out_channels
        stride = m.stride[0]
        padding = m.padding[0]
        n = feature_size[inchannel]
        fp32mut_this = 0
        fp32plus_this = 0
        #Conv
        outsize = math.floor((n + 2 * padding - kernel_size) / stride + 1)
        fp32mut_this += inchannel * outchennel * outsize**2*kernel_size**2
        fp32plus_this += inchannel * outchennel * outsize**2*(kernel_size**2-1)
        print('intput:',n,'*',n, 'output:',outsize,'*',outsize,'   ',inchannel,'to',outchennel)
        return fp32plus_this, fp32mut_this
    
def cal_Conv3d(m):
        print(type(m))
        kernel_size = m.kernel_size[1]
        inchannel =  m.in_channels
        outchennel = m.out_channels
        stride = m.stride[1]
        padding = m.padding[1]
        
        depth =  m.kernel_size[0]
        timestride = m.stride[0]
        timespadding = m.stride[0]
        
        n = feature_size[inchannel]
        fp32mut_this = 0
        fp32plus_this = 0
        #Conv
        outtime = feature_size_2[outchennel]
        outsize = math.floor((n + 2 * padding - kernel_size) / stride + 1)
        
        fp32mut_this += inchannel * outchennel * outsize**2*outtime * kernel_size**2*depth
        fp32plus_this += inchannel * outchennel * outsize**2*outtime * (kernel_size**2*depth-1)
        print('intput:',time_windows,'*',n,'*',n, 'output:',outtime,'*',outsize,'*',outsize,'   ',inchannel,'to',outchennel)
        return fp32plus_this, fp32mut_this


def cal_Linear(m):
        print(type(m))
        input_size = m.in_features
        hedden_size =  m.out_features
        #Conv
        fp32mut_this = 0
        fp32plus_this = 0
        fp32mut_this += input_size*input_size * input_size
        fp32plus_this += input_size*input_size * input_size
        print('intput:',input_size,'*',input_size, 'output:',hedden_size,'*',hedden_size)
        return fp32plus_this, fp32mut_this

isliaf = False
def cal_ops(model):
    fp32plus = 0
    fp32mut = 0
    for idx,m in enumerate(model.modules()):
        if isinstance(m,LIAFResNet) or isinstance(m,nn.Sequential):
            continue
        if isinstance(m,LIAFConvCell) :
            fp32plus_this,fp32mut_this = cal_LIAFConvCell(m,isliaf)
            print('fp32+=',fp32plus_this,'fp32*=',fp32mut_this)
        elif isinstance(m,Temporal_Conv2d):
            fp32plus_this,fp32mut_this = cal_Temporal_Conv2d(m,isliaf)
            print('fp32+=',fp32plus_this,'fp32*=',fp32mut_this)
        elif isinstance(m,Conv2d):
            fp32plus_this,fp32mut_this = cal_Conv2d(m)
            print('fp32+=',fp32plus_this,'fp32*=',fp32mut_this)
        elif isinstance(m,Linear):
            fp32plus_this,fp32mut_this = cal_Linear(m)
            print('fp32+=',fp32plus_this,'fp32*=',fp32mut_this)
        elif isinstance(m,Conv3d):
            fp32plus_this,fp32mut_this = cal_Conv3d(m)
            print('fp32+=',fp32plus_this,'fp32*=',fp32mut_this)
        else:
            continue
        fp32plus += fp32plus_this
        fp32mut += fp32mut_this
    return fp32plus , fp32mut


- LIAF18 fp32+= 12668145664 fp32*= 14159104000
- LIAF34 fp32+= 25783130112 fp32*= 28901059584
- 3D-18  fp32+= 12082982912 fp32*= 12492945408
- 3D-34  fp32+= 20671672320 fp32*= 21411969024
- LIF18  fp32+= 12668145664 fp32*= 268636160
- LIF34  fp32+= 25783130112 fp32*= 288479232
- 2D-18  fp32+= 1574797312 fp32*= 1770123264
- 2D-34  fp32+= 3211689984 fp32*= 3611627520

In [3]:

#fp32plus , fp32mut = cal_ops(snn)
#print(type(snn),'fp32+=',fp32plus,'fp32*=',fp32mut)
isliaf = False
fp32plus , fp32mut = cal_ops(snn18)
print(type(snn18),'fp32+=',fp32plus,'fp32*=',fp32mut)

<class 'LIAF.LIAFConvCell'>
intput: 224 * 224 output: 112 * 112
fp32+= 629407744 fp32*= 12845056
<class 'torch.nn.modules.conv.Conv2d'>
intput: 224 * 224 output: 112 * 112     2 to 64
fp32+= 77070336 fp32*= 78675968
<class 'LIAF.Temporal_Conv2d'>
intput: 55 * 55 output: 55 * 55
fp32+= 796083200 fp32*= 3097600
<class 'LIAF.Temporal_Conv2d'>
intput: 55 * 55 output: 55 * 55
fp32+= 796083200 fp32*= 3097600
<class 'LIAF.Temporal_Conv2d'>
intput: 55 * 55 output: 55 * 55
fp32+= 796083200 fp32*= 3097600
<class 'LIAF.Temporal_Conv2d'>
intput: 55 * 55 output: 55 * 55
fp32+= 796083200 fp32*= 3097600
<class 'torch.nn.modules.conv.Conv2d'>
intput: 55 * 55 output: 28 * 28     64 to 128
fp32+= 0 fp32*= 6422528
<class 'LIAF.Temporal_Conv2d'>
intput: 55 * 55 output: 28 * 28
fp32+= 412647424 fp32*= 1605632
<class 'LIAF.Temporal_Conv2d'>
intput: 28 * 28 output: 28 * 28
fp32+= 823689216 fp32*= 1605632
<class 'LIAF.Temporal_Conv2d'>
intput: 28 * 28 output: 28 * 28
fp32+= 823689216 fp32*= 1605632
<class 'LI

In [4]:
from torchvision.models import resnet34 as res34
from thop import profile
from thop import clever_format
model = res34()
input = torch.randn(1, 3, 224, 224)
macs, params = profile(model, inputs=(input, ))
macs, params = clever_format([macs, params], "%.3f")
print(macs)

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_bn() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[WARN] Cannot find rule for <class 'torchvision.models.resnet.BasicBlock'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.Sequential'>. Treat it as zero Macs and zero Params.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[WARN] Cannot find rule for <class 'torchvision.models.resnet.ResNet'>. Treat it as zero Macs and zero Params.
3.671G


In [5]:
print("Total number of paramerters in networks is {}  ".format(sum(x.numel() for x in snn18.parameters())))

Total number of paramerters in networks is 11693233  
